In [104]:
import requests
import json, sys, os
import pandas as pd
from datetime import datetime, date
from io import StringIO
from dateutil.rrule import rrule, MONTHLY

from tqdm.notebook import tqdm

In [231]:
list(rrule(MONTHLY, dtstart=date(2023, 1, 1), until=datetime.now().date()))

[datetime.datetime(2023, 1, 1, 0, 0),
 datetime.datetime(2023, 2, 1, 0, 0),
 datetime.datetime(2023, 3, 1, 0, 0)]

In [224]:
def transform_data(res, year, mo):
    d = StringIO(res.text)
    data = pd.read_csv(filepath_or_buffer=d)
    data['yearmo'] = int(f"{year}{mo}")
    return data

def extract():
    # big_dataframe = pd.DataFrame()
    # for dt in rrule(MONTHLY, dtstart=date(2022, 6, 1), until=datetime.now().date()):
    for refTime in tqdm(list(rrule(MONTHLY, dtstart=date(2012, 1, 1), until=datetime.now().date()))):
        refDateYr, refDateMo = refTime.strftime("%Y-%m").split("-")
            
        link = f"https://climate.weather.gc.ca/prods_servs/cdn_climate_summary_report_e.html?intYear={refDateYr}&intMonth={int(refDateMo)}&prov=&dataFormat=csv&btnSubmit=Download+data"
        res = requests.get(link)
        
        df = transform_data(res, refDateYr, refDateMo)
        load_data(df)
        # big_dataframe = pd.concat([big_dataframe, df])
    return df

def load_data(df):
    if not os.path.exists("weather_stations.csv"):
        # with open("weather_stations.csv", 'w') as file:
        df.to_csv("weather_stations.csv", mode='w', index=False, header=True)
            
    else:
        df.to_csv("weather_stations.csv", mode='a', index=False, header=False)

In [225]:
extract()

  0%|          | 0/135 [00:00<?, ?it/s]

,Long,Lat,Stn_Name,Clim_ID,Prov_or_Ter,Tm,DwTm,D,Tx,DwTx,...,DwP,P%N,S_G,Pd,BS,DwBS,BS%,HDD,CDD,yearmo
